In [1]:
#importing required libraries
import os
import numpy as np
import pandas as pd
from graphviz import Digraph

In [2]:
# Function to extract numerical parts from strings and calculate average
def calculate_average(row):
    numerical_values = []

    for value in row:
        if isinstance(value, str):
            parts = value.split()
            for part in parts:
                if part.isdigit():
                    numerical_values.append(int(part))
        elif isinstance(value, (int, float)) and not np.isnan(value):
            numerical_values.append(value)

    if numerical_values:
        return sum(numerical_values) / len(numerical_values)
    else:
        return np.nan

In [3]:
#Initialising DiGraph
graph = Digraph('G',format='pdf',engine='fdp',graph_attr={'size':'12,12','ratio':'fill','splines':'ortho','label':'Network Topology','fontsize':'32'})
ip_to_id = {}
unq_id = 1

In [4]:
#folders list and colors list
folders = ['BBC_UK','Global_Suzuki','Google','HDFC','Hyundai','MICROSOFT','OpenAI','Orange_Networks','Rakuten','Zoom']
fold_as = {}
colours = [
    "#1f77b4",  # Blue
    "#ff7f0e",  # Orange
    "#2ca02c",  # Green
    "#d62728",  # Red
    "#9467bd",  # Purple
    "#8c564b",  # Brown
    "#e377c2",  # Pink
    "#7f7f7f",  # Gray
    "#bcbd22",  # Olive
    "#17becf"   # Teal
]

In [5]:
#OpenAI destination
for j in range(len(folders)):
    folder_path = folders[j]
    file_list = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    for index in range(len(file_list)):
        file_path = os.path.join(folder_path, file_list[index])
        df = pd.read_csv(file_path,index_col=0)
    
        #Assigning random ASnumber if not present for source ip
        if index==0:
            df.iloc[0,8] = 'start1'
        if index==2:
            df.iloc[0,8] = 'start2'
    
        #Removing rows with private ip
        df = df.dropna(subset=['ASN_Number'])
        df = df.reset_index(drop=True)
    
        #creating new column average ping
        col_to_avg = ['1','2','3']
        df['Average'] = df[col_to_avg].apply(calculate_average, axis=1)
    
        #adding source node
        if df.iloc[0,8] not in ip_to_id:
            ip_to_id[df.iloc[0,8]] = unq_id
            graph.node(str(unq_id),label=file_list[index].split(".")[0],fontsize='5',image="images/source.png",imagescale='true',fixedsize='true',height='0.6',width='0.6')
            unq_id = unq_id+1
            
        #adding destination node
        if df.iloc[len(df.index)-1,8] not in ip_to_id:
            ip_to_id[df.iloc[len(df.index)-1,8]] = unq_id
            fold_as[folder_path] = folder_path+'('+'AS'+str(int(df.iloc[len(df.index)-1,8]))+')'
            graph.node(str(unq_id),label=folder_path,fontsize='6',image="images/destination.png",imagescale='true',fixedsize='true',height='0.8',width='0.8')
            unq_id = unq_id+1
        
        #adding intermediate nodes
        for i in range(1,len(df.index)-1):
            if df.iloc[i,8] not in ip_to_id:
                ip_to_id[df.iloc[i,8]] = unq_id
                lbl = 'AS'+str(int(df.iloc[i,8]))
                graph.node(str(unq_id),shape='box',label=lbl,fontsize='6',image="images/router.png",imagescale='true',fixedsize='true',height='0.512',width='0.534')
                unq_id = unq_id+1
            
        #adding edges between nodes
        for i in range(1,len(df.index)):
            if df.iloc[i,8]!=df.iloc[i-1,8]:
                src = str(ip_to_id[df.iloc[i,8]])
                tar = str(ip_to_id[df.iloc[i-1,8]])
                w = df.iloc[i,13]-df.iloc[i-1,13]
                if w<=0:#setting latency=1ms if negative
                    w = 12
                graph.edge(tar,src,weight=str(w),color=colours[j],arrowhead='open',arrowsize='0.25',penwidth='0.5')

In [6]:
# Define the legend node with the color mappings as an HTML-like table
legend = f"<<TABLE BORDER='0' CELLBORDER='1' CELLSPACING='0'><TR><TD COLSPAN='2'><B>Destination</B></TD></TR>"
for folder, color in zip(folders, colours):
    legend += f"<TR><TD BGCOLOR='{color}'>{fold_as[folder]}</TD></TR>"
legend += "</TABLE>>"

graph.node('legend', label=legend, shape='none', fontsize='10', margin='0.2,0.1')

#Rendering graph
graph.render('Visualisation', cleanup=True, view=True)

'Visualisation.pdf'